In [20]:
import newleg, inverse, xyzfield
import geofield
import numpy, scipy
import fibonacci_sphere
from matplotlib import pyplot

In [2]:
m,l=newleg.degrees(13)

In [121]:
from importlib import reload
reload(geofield); reload(newleg); reload(xyzfield)

<module 'xyzfield' from '/home/josecper/Programs/python/xyzfield.py'>

In [3]:
chaos=geofield.ChaosData("../data/CHAOS5X/gauss_coeff_chaos5x.dat")

In [4]:
gcomp=chaos.gcomp

In [8]:
order=13

In [18]:
gcomp=numpy.zeros(order*(order+2)+1)

In [99]:
gcomp[0,1]

-29658.132000000001

In [114]:
l

array([ 0,  1,  1,  1,  2,  2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  7,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,
        8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  9,  9,  9,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9, 10, 10,
       10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
       10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13,
       13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
       13, 13, 13, 13, 13, 13, 13, 13, 13])

In [106]:
theta,phi=fibonacci_sphere.grid(n=10001)

In [131]:
x,y,z=xyzfield.xyzfieldv2(gtest,phi,theta)

In [38]:
gcomp.shape

(224, 441)

In [ ]:
pyplot.show(xyzfield.xyzcontour(theta,phi,x,y,z,regular=False))

In [127]:
pyplot.show(xyzfield.xyzcontour(theta,phi,xp,yp,zp,regular=False))

/usr/lib/python3.5/site-packages/mpl_toolkits/basemap/__init__.py:3644: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  xx = x[x.shape[0]/2,:]


In [126]:
xp,yp,zp,leg,pleg=xyzfield.xyzfieldv(chaos.g[0,:,:],chaos.h[0,:,:],phi,theta,regular=False)

In [120]:
gcomp[0,10]

-2276.4965000000002

In [119]:
chaos.g[0,abs(m[11]),l[11]]

-2276.4965000000002

In [61]:
m[3],l[3]

(-1, 1)

In [129]:
gtest=numpy.zeros(196)

In [130]:
gtest[1]=-30000

In [142]:
m

array([  0,   0,   1,  -1,   0,   1,  -1,   2,  -2,   0,   1,  -1,   2,
        -2,   3,  -3,   0,   1,  -1,   2,  -2,   3,  -3,   4,  -4,   0,
         1,  -1,   2,  -2,   3,  -3,   4,  -4,   5,  -5,   0,   1,  -1,
         2,  -2,   3,  -3,   4,  -4,   5,  -5,   6,  -6,   0,   1,  -1,
         2,  -2,   3,  -3,   4,  -4,   5,  -5,   6,  -6,   7,  -7,   0,
         1,  -1,   2,  -2,   3,  -3,   4,  -4,   5,  -5,   6,  -6,   7,
        -7,   8,  -8,   0,   1,  -1,   2,  -2,   3,  -3,   4,  -4,   5,
        -5,   6,  -6,   7,  -7,   8,  -8,   9,  -9,   0,   1,  -1,   2,
        -2,   3,  -3,   4,  -4,   5,  -5,   6,  -6,   7,  -7,   8,  -8,
         9,  -9,  10, -10,   0,   1,  -1,   2,  -2,   3,  -3,   4,  -4,
         5,  -5,   6,  -6,   7,  -7,   8,  -8,   9,  -9,  10, -10,  11,
       -11,   0,   1,  -1,   2,  -2,   3,  -3,   4,  -4,   5,  -5,   6,
        -6,   7,  -7,   8,  -8,   9,  -9,  10, -10,  11, -11,  12, -12,
         0,   1,  -1,   2,  -2,   3,  -3,   4,  -4,   5,  -5,   

In [145]:
cossin = numpy.zeros((len(phi), 13*(13+2)+1))

In [157]:
cossin[:, m < 0] = numpy.sin(phi)

ValueError: shape mismatch: value array of shape (10001,) could not be broadcast to indexing result of shape (91,10001)

array([ -1,  -1,  -2,  -1,  -2,  -3,  -1,  -2,  -3,  -4,  -1,  -2,  -3,
        -4,  -5,  -1,  -2,  -3,  -4,  -5,  -6,  -1,  -2,  -3,  -4,  -5,
        -6,  -7,  -1,  -2,  -3,  -4,  -5,  -6,  -7,  -8,  -1,  -2,  -3,
        -4,  -5,  -6,  -7,  -8,  -9,  -1,  -2,  -3,  -4,  -5,  -6,  -7,
        -8,  -9, -10,  -1,  -2,  -3,  -4,  -5,  -6,  -7,  -8,  -9, -10,
       -11,  -1,  -2,  -3,  -4,  -5,  -6,  -7,  -8,  -9, -10, -11, -12,
        -1,  -2,  -3,  -4,  -5,  -6,  -7,  -8,  -9, -10, -11, -12, -13])